In [1]:
import torch
import numpy as np
from stable_baselines3 import SAC

In [ ]:
class Net(torch.nn.Module):

  def __init__(self, n_fc1, n_fc2, activation,n_layers,**kwargs):
    super(Net, self).__init__()

    # Unpack the dictionary
    self.args     = kwargs
    self.dtype    = torch.float
    self.use_cuda = torch.cuda.is_available()
    self.device   = torch.device("cpu")

    self.input_size = 6 #State size: Ca, T, Ca setpoint and T setpoint
    self.output_sz  = 7 #Output size: Reactor Ks size
    self.n_layers = torch.nn.ModuleList()
    self.hs1        = n_fc1                                    # !! parameters
    self.hs2        = n_fc2                                      # !! parameter

    # defining layer
    self.hidden1 = torch.nn.Linear(self.input_size, self.hs1 )
    self.act = activation()
    self.hidden2 = torch.nn.Linear(self.hs1, self.hs2)
    for i in range(0,n_layers):
      linear_layer = torch.nn.Linear(self.hs2,self.hs2)
      self.n_layers.append(linear_layer)
    self.output = torch.nn.Linear(self.hs2, self.output_sz)

    #self.af     = torch.nn.Tanh()

  def forward(self, x):

    x = x.float()
    y           = self.act(self.hidden1(x))
    y           = self.act(self.hidden2(y))
    for i,layer in enumerate(self.n_layers):
      y           = self.act(layer(y))
    y           = F.relu6(self.output(y))                                       # range (0,6)

    return y